Import Libraries

In [1]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Dict, Optional, cast
from torch import Tensor
from collections import OrderedDict 

Build New Model

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
from PIL import Image
import cv2
import torchvision.transforms as transforms
import collections
import random
from random import seed, choice, sample
import os
import json
from google.colab.patches import cv2_imshow
import sys
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False
from os.path import exists, join, basename, splitext
import csv
import glob

In [4]:
%matplotlib inline
import os
import sys
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import skimage.transform
from PIL import Image
from collections import Counter
from tqdm.notebook import tqdm

import torch
torch.cuda.empty_cache()
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from nltk.translate.bleu_score import corpus_bleu

import warnings
warnings.filterwarnings("ignore")

In [5]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from gensim.models.fasttext import FastText
 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
os.chdir('/content/drive/My Drive/Final Year Project/')

In [7]:
print(os.path.abspath('.'))

/content/drive/.shortcut-targets-by-id/1sz5w7NJWd5pUCRXsoinjyyZ-EUMh1tbM/Final Year Project


In [8]:
filename = '/content/drive/My Drive/Final Year Project/input_images_train.txt'
read_file = open(filename,"r") 

train_image_paths = []
train_image_names = []
#print("Output of Read function is ")
data_read = read_file.read()
data_into_list = data_read.split("\n")
#print(data_into_list)
for i in range(len(data_into_list)):
    if(len(data_into_list[i])>1):
        train_image_paths.append(data_into_list[i])
        path = data_into_list[i]
        name_list = path.split("/")
        name = name_list[len(name_list)-1]
        #print(path)
        train_image_names.append(name)
print("No. of images: ",len(train_image_paths), len(train_image_names))

No. of images:  23431 23431


In [9]:
filename = '/content/drive/My Drive/Final Year Project/input_images_val.txt'
read_file = open(filename,"r") 

val_image_paths = []
val_image_names = []
#print("Output of Read function is ")
data_read = read_file.read()
data_into_list = data_read.split("\n")
#print(data_into_list)
for i in range(len(data_into_list)):
    if(len(data_into_list[i])>1):
        val_image_paths.append(data_into_list[i])
        path = data_into_list[i]
        name_list = path.split("/")
        name = name_list[len(name_list)-1]
        #print(path)
        val_image_names.append(name)
print("No. of images: ",len(val_image_paths), len(val_image_names))

No. of images:  7750 7750


In [10]:
# sets device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# # YOLO helper functions
# # define helper functions
import linecache

def get_words(file_name):
  # print(file_name)
  words = []

  N = get_total_no_lines(file_name)

  # print("Number of lines = ",N)

  line_number = 0

  with open (file_name, "r") as BigFile:
    data=BigFile.readlines()

  while(line_number < N):
    line = data[line_number]
    line_words = line.split()
    if(line_words[0]!="image"):
      break
    # print(line_words)
    obj_index = 4
    cur_words=[]
    if(line_words[obj_index]!="detections)"):
      while(obj_index < len(line_words)):
        cur_word = line_words[obj_index]
        if(any(chr.isdigit() for chr in cur_word)==False):
          if(cur_word[-1]==','):
            cur_words.append(line_words[obj_index][0:-1])
          else:
            cur_words.append(line_words[obj_index])
        obj_index = obj_index+1
    words.append(cur_words)
    line_number = line_number+1
  
  # print(words)
  # print("Words size = ",len(words))
  return words

def get_total_no_lines(file_name):
  file = open(file_name,"r")
  counter = 0
    
  Content = file.read()
  CoList = Content.split("\n")
  file.close()
    
  for i in CoList:
      if i:
          counter += 1
  
  return counter

In [12]:
#-----------------------new YOLOv5-----------------------------------------
#yolo Model
import sys
import contextlib
def yolo_obj_det(img_paths):
    yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    # Inference
    yolo_results = yolo(img_paths)
    #print("in yolo, calling functions")
    file_path = '/content/drive/My Drive/Final Year Project/yolov5/words_detected.txt'
    myoutputfile = open(file_path, "w+")
    with contextlib.redirect_stdout(myoutputfile):
      yolo_results.print()
    myoutputfile.close()
    word_array_det = get_words('/content/drive/My Drive/Final Year Project/yolov5/words_detected.txt')
    print("Final word_array_det = ",word_array_det)
    print(len(word_array_det))
    # Defining values for parameters
    embedding_size = 300
    window_size = 5
    min_word = 5
    down_sampling = 1e-2

    fast_Text_model_det = FastText(word_array_det,
                              size=embedding_size,
                              window=window_size,
                              min_count=1,
                              sample=down_sampling,
                              workers = 2,
                              sg=1,
                              iter=100)

    features_det=[]
    for i in range(len(word_array_det)):
      ft2=np.empty((300))
      for word in word_array_det[i]:
        ft2+=fast_Text_model_det.wv[word]
      #print(ft2.shape)
      #print(ft2)
      features_det.append(ft2)

    for i in range(len(features_det)):
      ft2 = features_det[i]
      #print(ft2.shape)
      ft2 = ft2.reshape(1,300)
      ft2_out = np.tile(ft2, (14,14,1,1))
      #print(ft2_out.shape)
      ft2_out = np.transpose(ft2_out, [2, 3, 0, 1])
      #print(ft2_out.shape)
      features_det[i] = ft2_out

    return word_array_det,features_det

In [13]:
image_paths = []
image_names = []

for i in range(50):
  image_paths.append(train_image_paths[i])
  image_names.append(train_image_names[i])

print(len(image_paths), len(image_names))

50 50


In [ ]:
word_array_det, features_det = yolo_obj_det(train_image_paths)
print(len(word_array_det), len(features_det))

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-5-20 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [16]:
path = '/content/drive/My Drive/Final Year Project/yolo_features/'
for i in range(len(features_det)):
  name = train_image_names[i]
  name = name[:len(name)-4]
  feature = features_det[i]
  np_path = path + name
  np.save(np_path, feature)

In [17]:
path = '/content/drive/My Drive/Final Year Project/yolo_features/'
for i in range(len(image_paths)):
  name = image_names[i]
  name = name[:len(name)-4]
  np_path = path + name + '.npy'
  out = np.load(np_path)
  print(out.shape, out.dtype)

(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
(1, 300, 14, 14) float64
